In [1]:
!pip install pandas

In [2]:
import pandas as pd
import re
import ast
import json
from collections import Counter
from dateutil import parser
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import *
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import random
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
import pickle
from sklearn.feature_extraction.text import CountVectorizer
import joblib
pd.options.mode.chained_assignment = "warn"
from sklearn.naive_bayes import MultinomialNB
import scipy.sparse
import nltk
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
import spacy


from tqdm import tqdm
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Anton\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Anton\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Anton\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
df = pd.read_csv('H:\Python311\Scripts\Файлы\drive-download-20240225T125924Z-002/news_after_preprocessing.csv', sep=';')

In [5]:
df

,url,date and time,tag,title,text,most_popular_tag,preprocessed_text
0,https://www.kommersant.ru/doc/973901,2004-04-22 17:39:00,Мир,Абашидзе упразднил МГБ Аджарии,Глава Аджарии Аслан Абашидзе принял решение об...,Мир,глава аджарии аслан абашидзе принял решение уп...
1,https://www.kommersant.ru/doc/2200895,2013-05-30 18:15:00,Общество,РЖД будут продавать билеты через «Почту России»,С 1 июля Российские железные дороги начнут про...,Общество,июля российские железные дороги начнут продава...
2,https://www.kommersant.ru/doc/3859122,2019-01-18 20:48:00,Финансы,Объем операций в биткойнах в даркнете вырос вдвое,Использование биткойнов для оплаты товаров и у...,Финансы,использование биткойнов оплаты товаров услуг д...
3,https://www.kommersant.ru/doc/1171617,2009-05-17 19:39:00,Общество,Япония и Великобритания сообщили о новых случа...,Число официально подтвержденных случаев зараже...,Общество,число официально подтвержденных случаев зараже...
4,https://www.kommersant.ru/doc/2415833,2014-02-24 20:50:00,Политика,Владимир Путин обсудил с Александром Лукашенко...,Президент России Владимир Путин по телефону об...,Политика,президент россии владимир путин телефону обсуд...
...,...,...,...,...,...,...,...
1271190,https://www.fontanka.ru/2023/11/19/72929774/,2023-11-19 00:00:00,"['Погода в Петербурге', 'Погода', 'Осень', 'Ан...",Влияние антициклона ослабевает. В Петербурге т...,Облачная с прояснениями погода ожидается в Пет...,Погода,облачная прояснениями погода ожидается петербу...
1271191,https://www.fontanka.ru/2023/11/14/72912443/,2023-11-14 00:00:00,"['МВД', 'Полиция Петербурга', 'Задержание поли...",Жажда унизить,По медиапросторам разошлось видео задержания н...,МВД,медиапросторам разошлось видео задержания нача...
1271192,https://www.fontanka.ru/2023/11/19/72930512/,2023-11-19 00:00:00,"['Дальнобойщики', 'Забастовка', 'Польша', 'Укр...",Тысячи грузовиков на границе с Украиной: в Пол...,На польской стороне границы с Украиной застрял...,Украина,польской стороне границы украиной застряли мог...
1271193,https://www.fontanka.ru/2023/11/19/72930443/,2023-11-19 00:00:00,"['Владимир Путин', 'Саммит G20', 'Саммит']",Владимир Путин примет участие в виртуальном са...,Президент России Владимир Путин на следующей н...,Владимир Путин,президент россии владимир путин следующей неде...


In [6]:
df = df[~df['preprocessed_text'].isnull()]

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1271191 entries, 0 to 1271194
Data columns (total 7 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   url                1271191 non-null  object
 1   date and time      1271191 non-null  object
 2   tag                1271191 non-null  object
 3   title              1271191 non-null  object
 4   text               1271191 non-null  object
 5   most_popular_tag   1271191 non-null  object
 6   preprocessed_text  1271191 non-null  object
dtypes: object(7)
memory usage: 77.6+ MB


In [7]:
import sys
!{sys.executable} -m pip install spacy

In [17]:
pip install spacy-udpipe

   ---------------------------------------- 0.0/892.6 kB ? eta -:--:--
   ---------------------------------------- 10.2/892.6 kB ? eta -:--:--
   ---------------------------------------- 10.2/892.6 kB ? eta -:--:--
   - ------------------------------------- 30.7/892.6 kB 220.2 kB/s eta 0:00:04
   - ------------------------------------- 41.0/892.6 kB 219.4 kB/s eta 0:00:04
   -- ------------------------------------ 61.4/892.6 kB 273.8 kB/s eta 0:00:04
   --- ----------------------------------- 71.7/892.6 kB 245.8 kB/s eta 0:00:04
   ---- ---------------------------------- 92.2/892.6 kB 275.8 kB/s eta 0:00:03
   ---- --------------------------------- 112.6/892.6 kB 298.5 kB/s eta 0:00:03
   ------ ------------------------------- 143.4/892.6 kB 341.3 kB/s eta 0:00:03
   ------ ------------------------------- 153.6/892.6 kB 328.1 kB/s eta 0:00:03
   ------- ------------------------------ 174.1/892.6 kB 338.5 kB/s eta 0:00:03
   -------- ----------------------------- 204.8/892.6 kB 355.7 kB

In [23]:
!pip install -U pip setuptools wheel

  Using cached setuptools-69.1.1-py3-none-any.whl.metadata (6.2 kB)
  Using cached wheel-0.42.0-py3-none-any.whl.metadata (2.2 kB)
Using cached setuptools-69.1.1-py3-none-any.whl (819 kB)
Using cached wheel-0.42.0-py3-none-any.whl (65 kB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 65.5.0
    Uninstalling setuptools-65.5.0:
      Successfully uninstalled setuptools-65.5.0


In [24]:
!pip install -U spacy

In [25]:
!python -m spacy download ru_core_news_sm

     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     --------------------------------------- 0.0/15.3 MB 325.1 kB/s eta 0:00:47
     --------------------------------------- 0.0/15.3 MB 281.8 kB/s eta 0:00:54
     --------------------------------------- 0.1/15.3 MB 459.5 kB/s eta 0:00:34
     --------------------------------------- 0.1/15.3 MB 459.5 kB/s eta 0:00:34
     --------------------------------------- 0.1/15.3 MB 327.7 kB/s eta 0:00:47
     --------------------------------------- 0.1/15.3 MB 359.9 kB/s eta 0:00:43
     --------------------------------------- 0.1/15.3 MB 370.8 kB/s eta 0:00:41
     --------------------------------------- 0.2/15.3 MB 378.3 kB/s eta 0:00:40
     --------------------------------------- 0.2/15.3 MB 406.9 kB/s eta 0:00:38
      -------------------------------------- 0.2/15.3 MB 389.1 k

In [8]:
import spacy

In [31]:
!python -m spacy download ru_core_news_md

     ---------------------------------------- 0.0/41.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/41.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/41.9 MB ? eta -:--:--
     --------------------------------------- 0.0/41.9 MB 435.7 kB/s eta 0:01:37
     --------------------------------------- 0.0/41.9 MB 326.8 kB/s eta 0:02:09
     --------------------------------------- 0.1/41.9 MB 363.1 kB/s eta 0:01:56
     --------------------------------------- 0.1/41.9 MB 381.3 kB/s eta 0:01:50
     --------------------------------------- 0.1/41.9 MB 381.3 kB/s eta 0:01:50
     --------------------------------------- 0.1/41.9 MB 291.5 kB/s eta 0:02:24
     --------------------------------------- 0.1/41.9 MB 297.7 kB/s eta 0:02:21
     --------------------------------------- 0.1/41.9 MB 340.5 kB/s eta 0:02:03
     --------------------------------------- 0.2/41.9 MB 350.7 kB/s eta 0:01:59
     --------------------------------------- 0.2/41.9 MB 337.8 k

In [9]:
nlp = spacy.load("ru_core_news_md")

In [10]:
print(nlp.pipe_names)

['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [11]:
docs = list(nlp.pipe(df['text'][:10], n_process=5, batch_size=2))

In [14]:
docs[1]

С 1 июля Российские железные дороги начнут продавать билеты в офисах «Почты России», заявил на пресс-конференции в Петербурге заместитель начальника департамента пассажирских сообщений ОАО «РЖД» Дмитрий Корней. Проект коснется не всех городов, отмечает «Фонтанка». «Там, где в связи с малыми объемами продаж кассы уже не работают, в отделениях почтовой связи можно будет приобрести электронный билет. И, не распечатывая бланк заказа, не проходя процедуру так называемой регистрации, сесть в поезд. При себе пассажир должен будет иметь только бланк, который выдадут на почте», — уточнил первый замначальника Северо-Западного филиала ОАО «Федеральная пассажирская компания» Станислав Зотин. Всего проект стартует в 24 городах.
Как пытаются улучшить работу «Почту России» — читайте в материале «Ъ» «Большие перемены». Об инновациях в РЖД читайте в статье «Ъ» «Чемодан—вокзал—досмотр».

In [15]:
def lemmatize(doc):
    words = []
    for token in doc:
        if (token.is_stop != True) and (token.is_punct != True) and\
            (token.is_space != True) and (token.is_digit != True):
            words.append(token.lemma_)
    return ' '.join(words)

In [26]:
processed_texts = []
for text in tqdm(df['text'], desc="Lemmatizing"):
    doc = nlp(text)
    processed_texts.append(lemmatize(doc))

Lemmatizing: 100%|████████████████████████████████████████████████████████| 1271191/1271191 [26:08:23<00:00, 13.51it/s]


In [27]:
df1 = pd.DataFrame({'processed_text': processed_texts})

In [29]:
df1.to_csv('H:/Python311/Scripts/Файлы/drive-download-20240225T125924Z-002/preprocessed_text.csv', index=False)

In [33]:
df1.iloc[1270000:]

,processed_text
1270000,гуп мосгортранс запустить праздник пасха росси...
1270001,фотография скончаться лидер кубинский революци...
1270002,российский федерация оказывать донецкий луганс...
1270003,американский союз защита гражданский свобода п...
1270004,лидер партия лдпр владимир жириновский прокомм...
...,...
1271186,облачная прояснениями погода ожидаться петербу...
1271187,медиапросторам разойтись видео задержание нача...
1271188,польский сторона граница украина застрять выех...
1271189,президент россия владимир путин следующий неде...


In [3]:
df1 = pd.read_csv('H:/Python311/Scripts/Файлы/drive-download-20240225T125924Z-002/preprocessed_text.csv', sep=',')

In [14]:
df1['most_popular_tag'] = df['most_popular_tag']

In [15]:
df1.head()

,processed_text,most_popular_tag
0,глава аджария аслан абашидзе принять решение у...,Мир
1,июль российский железный дорога начать продава...,Общество
2,использование биткойнов оплата товар услуга да...,Финансы
3,число официально подтвердить случай заражение ...,Общество
4,президент россия владимир путин телефон обсуди...,Политика


In [16]:
df1.to_csv('H:/Python311/Scripts/Файлы/drive-download-20240225T125924Z-002/preprocessed_text_and_tag.csv', sep=';', index = False)

In [17]:
df1.head()

,processed_text,most_popular_tag
0,глава аджария аслан абашидзе принять решение у...,Мир
1,июль российский железный дорога начать продава...,Общество
2,использование биткойнов оплата товар услуга да...,Финансы
3,число официально подтвердить случай заражение ...,Общество
4,президент россия владимир путин телефон обсуди...,Политика


In [10]:
lemmatizer = WordNetLemmatizer()
tx = lemmatizer.lemmatize(df.text[:2])

TypeError: unhashable type: 'Series'

In [18]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,1))
tfidf_vectorizer_text = tfidf_vectorizer.fit(df1['processed_text'])

In [21]:
# Преобразование текста в векторы TF-IDF
tfidf_vectors = tfidf_vectorizer_text.transform(df1['processed_text'])

In [30]:
tfidf_vectors[:50].shape[0]

50

In [22]:
# Создание DataFrame с TF-IDF векторами
tfidf_df = pd.DataFrame(tfidf_vectors.toarray(), columns=tfidf_vectorizer_text.get_feature_names_out())

MemoryError: Unable to allocate 8.75 TiB for an array with shape (1271191, 945859) and data type float64

In [33]:
import numpy as np

# Путь к CSV файлу, куда будут добавляться данные
csv_path = 'H:/Python311/Scripts/Файлы/drive-download-20240225T125924Z-002/tfidf_vectors_batch.csv'

# Размер батча
batch_size = 1

# Количество строк
num_rows = tfidf_vectors[:5].shape[0]

# Количество батчей
num_batches = (num_rows - 1) // batch_size + 1

# Цикл по всем батчам
for i in range(num_batches):
    # Определение индексов текущего батча
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, num_rows)
    
    # Получение TF-IDF векторов для текущего батча
    tfidf_vectors_batch = tfidf_vectors[start_idx:end_idx]
    
    # Преобразование разреженной матрицы в плотную матрицу numpy
    dense_batch = tfidf_vectors_batch.toarray()
    
    # Добавление батча в CSV файл
    if i == 0:
        # Если это первый батч, записываем его с заголовком
        pd.DataFrame(dense_batch, columns = tfidf_vectorizer.get_feature_names_out()).to_csv(csv_path, mode='a', header=True, index=False)
    else:
        # Если это последующие батчи, записываем их без заголовка
        pd.DataFrame(dense_batch).to_csv(csv_path, mode='a', header=False, index=False)


In [34]:
# Чтение данных из CSV файла
# Для чтения батчей вам также потребуется использовать цикл
# Пример чтения первых 1000 строк из файла
first_batch = pd.read_csv(csv_path, nrows=5)

In [76]:
first_batch.columns[700041]

'процкий'

In [ ]:
'''
После анализа матрицы, полученной после векторизатора, видим, что в нее попадает много признаков, включая непонятные символы.
Необходимо сократить количество признаков до ТОП-n и убрать редкие слова
'''

In [79]:
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")

In [87]:
# Настройка параметров векторизатора TF-IDF
tfidf_vectorizer_v2 = TfidfVectorizer(
    min_df=3000,                 # Минимальная частота токена в документах
    max_df=0.65,               # Максимальная частота токена в документах
    ngram_range=(1, 1),       # Диапазон n-грамм
    stop_words=russian_stopwords,     # Список стоп-слов для исключения
    max_features=50000         # Максимальное количество признаков (слов)
)

In [88]:
tfidf_vectorizer_text_v2 = tfidf_vectorizer_v2.fit(df1['processed_text'])

In [90]:
# Преобразование текста в векторы TF-IDF
tfidf_vectors_v2 = tfidf_vectorizer_text_v2.transform(df1['processed_text'])

In [91]:
tfidf_vectors_v2.shape

(1271191, 4793)

In [92]:
import numpy as np

# Путь к CSV файлу, куда будут добавляться данные
csv_path = 'H:/Python311/Scripts/Файлы/drive-download-20240225T125924Z-002/tfidf_vectors_batch_v2.csv'

# Размер батча
batch_size = 10000

# Количество строк
num_rows = tfidf_vectors_v2.shape[0]

# Количество батчей
num_batches = (num_rows - 1) // batch_size + 1

# Цикл по всем батчам
for i in tqdm(range(num_batches), desc="Processing batches"):
    # Определение индексов текущего батча
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, num_rows)
    
    # Получение TF-IDF векторов для текущего батча
    tfidf_vectors_batch_v2 = tfidf_vectors_v2[start_idx:end_idx]
    
    # Преобразование разреженной матрицы в плотную матрицу numpy
    dense_batch_v2 = tfidf_vectors_batch_v2.toarray()
    
    # Добавление батча в CSV файл
    if i == 0:
        # Если это первый батч, записываем его с заголовком
        pd.DataFrame(dense_batch_v2, columns = tfidf_vectorizer_v2.get_feature_names_out()).to_csv(csv_path, mode='a', header=True, index=False)
    else:
        # Если это последующие батчи, записываем их без заголовка
        pd.DataFrame(dense_batch_v2).to_csv(csv_path, mode='a', header=False, index=False)

Processing batches: 100%|████████████████████████████████████████████████████████████| 128/128 [46:31<00:00, 21.81s/it]


In [93]:
df_tfidf = pd.read_csv(csv_path, nrows=50000)

In [94]:
df_tfidf

,00,05,07,08,09,10,100,11,12,13,...,якутия,январь,яндекс,янукович,япония,японский,яркий,ярославский,ясный,ящик
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.063703,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.111497,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
49996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
49997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
49998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.051169,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [99]:
df1[:50000]

,processed_text,most_popular_tag
0,глава аджария аслан абашидзе принять решение у...,Мир
1,июль российский железный дорога начать продава...,Общество
2,использование биткойнов оплата товар услуга да...,Финансы
3,число официально подтвердить случай заражение ...,Общество
4,президент россия владимир путин телефон обсуди...,Политика
...,...,...
49995,президент грузия михаил саакашвили призвать на...,Мир
49996,россия обладать достоверный данными иметь мест...,Мир
49997,министерство финансов предлагать гг. ежегодно ...,Бизнес
49998,федеральный медико биологический агентство фмб...,Пандемия коронавируса


In [96]:
filtered_columns = df_tfidf.filter(regex='[^0-9]+', axis=1)

In [97]:
filtered_columns

,abc,afp,agence,air,airbus,airlines,al,and,android,ap,...,якутия,январь,яндекс,янукович,япония,японский,яркий,ярославский,ясный,ящик
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.063703,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.111497,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
49996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
49997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
49998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.051169,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [100]:
# Объединение TF-IDF DataFrame с исходным DataFrame
df_res = pd.concat([df1['most_popular_tag'][:50000], filtered_columns], axis=1)

# Сохранение результата в CSV файл
df_res.to_csv('H:/Python311/Scripts/Файлы/drive-download-20240225T125924Z-002/preprocessed_text_and_tag_with_tfidf_v2.csv', sep=';', index=False)

In [101]:
df_res.head()

,most_popular_tag,abc,afp,agence,air,airbus,airlines,al,and,android,...,якутия,январь,яндекс,янукович,япония,японский,яркий,ярославский,ясный,ящик
0,Мир,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,Общество,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,Финансы,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.063703,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,Общество,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.111497,0.0,0.0,0.0,0.0,0.0
4,Политика,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [19]:
import joblib

In [20]:
joblib.dump(tfidf_vectorizer_text, 'H:/Python311/Scripts/Файлы/drive-download-20240225T125924Z-002/tfidf_vectorizer_model_new.joblib')

['H:/Python311/Scripts/Файлы/drive-download-20240225T125924Z-002/tfidf_vectorizer_model_new.joblib']

In [ ]:
# Загрузка модели
loaded_tfidf_vectorizer = joblib.load(model_path)

In [ ]:
tfidf_vectors = loaded_tfidf_vectorizer.transform(df1['processed_text'])

In [ ]:
tfidf_vectorizer_text_transform = tfidf_vectorizer_text.transform

In [98]:
tfidf_vectorizer_text.shape

(1271191, 1500)

In [99]:
tfidf_vectorizer.get_feature_names_out().shape

(1500,)

In [ ]:
pca = PCA()

# Применение PCA к TF-IDF матрице
pca.fit(tfidf_vectorizer_text.toarray())

# Построение кривой объясненной дисперсии
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Число компонент')
plt.ylabel('Объясненная дисперсия')
plt.title('Кривая объясненной дисперсии')
plt.grid(True)
plt.show()

In [95]:
# #Сохранение векторайзера
joblib.dump(tfidf_vectorizer, 'H:/Python311/Scripts/Файлы/drive-download-20240225T125924Z-002/tfidf_vectorizer.pkl')

# # Сохранение разреженной матрицы в файл
joblib.dump(tfidf_vectorizer_text, 'H:/Python311/Scripts/Файлы/drive-download-20240225T125924Z-002/tfidf_vectorizer_text.pkl')

# # Преобразование разреженной матрицы в DataFrame
# df_vectors_1 = pd.DataFrame(count_vectorizer_text.toarray(), columns=count_vectorizer.get_feature_names_out())
# # Запись DataFrame в CSV файл
# df_vectors_1.to_csv('H:\Python311\Scripts\Файлы\drive-download-20240225T125924Z-002/countvectors_1.csv', index=False)

['H:/Python311/Scripts/Файлы/drive-download-20240225T125924Z-002/tfidf_vectorizer_text.pkl']

In [103]:
tfidf_vectorizer_text = joblib.load('H:/Python311/Scripts/Файлы/drive-download-20240225T125924Z-002/tfidf_vectorizer_text.pkl')

In [100]:
df_vectors = pd.DataFrame.sparse.from_spmatrix(tfidf_vectorizer_text, columns=tfidf_vectorizer.get_feature_names_out())

In [101]:
df_vectors.head()

,afp,apple,associated,bbc,bloomberg,covid,daily,facebook,google,group,...,этому,эфире,южной,юрий,является,являются,якобы,январе,января,японии
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.090733,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.158716
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [102]:
df_vectors.to_csv('H:/Python311/Scripts/Файлы/drive-download-20240225T125924Z-002/tfidf_vectors_text.csv', index=False)

In [110]:
type(df_vectors)

pandas.core.frame.DataFrame

In [79]:
df_vectors = pd.DataFrame(tfidf_vectorizer_text.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

MemoryError: Unable to allocate 12.7 TiB for an array with shape (1271191, 1374633) and data type float64

In [26]:
count_vectorizer = CountVectorizer()
count_vectorizer_text = count_vectorizer.fit_transform(df['preprocessed_text'])

# #Сохранение векторайзера
# joblib.dump(count_vectorizer, 'H:/Python311/Scripts/Файлы/drive-download-20240225T125924Z-002/count_vectorizer_1.pkl')

# # Сохранение разреженной матрицы в файл
# joblib.dump(count_vectorizer_text, 'H:/Python311/Scripts/Файлы/drive-download-20240225T125924Z-002/count_vectorizer_text_1.pkl')

# # Преобразование разреженной матрицы в DataFrame
# df_vectors_1 = pd.DataFrame(count_vectorizer_text.toarray(), columns=count_vectorizer.get_feature_names_out())
# # Запись DataFrame в CSV файл
# df_vectors_1.to_csv('H:\Python311\Scripts\Файлы\drive-download-20240225T125924Z-002/countvectors_1.csv', index=False)

In [8]:
# count_vectorizer_text = joblib.load('count_vectorizer_text_1.pkl')

In [47]:
# # Преобразование разреженной матрицы в DataFrame
# df_vectors_1 = pd.DataFrame.sparse.from_spmatrix(count_vectorizer_text, columns=count_vectorizer.get_feature_names_out())

In [ ]:
# Запись DataFrame в CSV файл
# df_vectors_1.to_csv('countvectors_1.csv', index=False)

In [112]:
# df_vectors_1 = pd.read_csv('/content/drive/MyDrive/Project/countvectors_1.csv')
# count_vectorizer_1 = joblib.load('/content/drive/MyDrive/Project/count_vectorizer_1.pkl')
# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(df_vectors, df.most_popular_tag, test_size=0.2, random_state=42)

# Создание и обучение модели наивного Байесовского классификатора с использованием батчей и отслеживанием прогресса
model = MultinomialNB()
batch_size = 32
total_batches = len(X_train) // batch_size + 1
unique_tags = df['most_popular_tag'].unique().tolist()

with tqdm(total=total_batches) as pbar:
    for i in range(0, len(X_train), batch_size):
        X_batch = X_train.iloc[i:i+batch_size]
        y_batch = y_train.iloc[i:i+batch_size]
        model.partial_fit(X_batch, y_batch, classes=unique_tags)
        pbar.update(1)  # Обновление прогресс-бара для каждого пройденного батча

# Оценка качества модели на тестовом наборе
accuracy = model.score(X_test, y_test)
print("Accuracy:", accuracy)

100%|██████████████████████████████████████████████████████████████████████████| 31780/31780 [1:33:30<00:00,  5.66it/s]


Accuracy: 0.5484406404996873


In [114]:
from sklearn.ensemble import RandomForestClassifier

# Создание и обучение модели Random Forest
model = RandomForestClassifier()
batch_size = 32
total_batches = len(X_train) // batch_size + 1
unique_tags = df['most_popular_tag'].unique().tolist()

with tqdm(total=total_batches) as pbar:
    for i in range(0, len(X_train), batch_size):
        X_batch = X_train.iloc[i:i+batch_size]
        y_batch = y_train.iloc[i:i+batch_size]
        model.fit(X_batch, y_batch)  # Обучение модели на текущем батче
        pbar.update(1)  # Обновление прогресс-бара для каждого пройденного батча

# Оценка качества модели на тестовом наборе
accuracy = model.score(X_test, y_test)
print("Accuracy:", accuracy)


100%|██████████████████████████████████████████████████████████████████████████| 31780/31780 [2:58:12<00:00,  2.97it/s]


Accuracy: 0.21285876675097054


In [4]:
df_res = pd.read_csv('H:/Python311/Scripts/Файлы/drive-download-20240225T125924Z-002/preprocessed_text_and_tag_with_tfidf_v2.csv', sep=';')

In [5]:
y = df_res.iloc[:, 0]
X = df_res.iloc[:, 1:]

In [6]:
y

0                          Мир
1                     Общество
2                      Финансы
3                     Общество
4                     Политика
                 ...          
49995                      Мир
49996                      Мир
49997                   Бизнес
49998    Пандемия коронавируса
49999                 Общество
Name: most_popular_tag, Length: 50000, dtype: object

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [8]:
X_test[:5]

,abc,afp,agence,air,airbus,airlines,al,and,android,ap,...,якутия,январь,яндекс,янукович,япония,японский,яркий,ярославский,ясный,ящик
33553,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9427,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12447,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39489,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.138621,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# Создание и обучение модели
model = GradientBoostingClassifier()
batch_size = 5000

total_batches = len(X_train) // batch_size + 1
unique_tags = y.unique().tolist()

with tqdm(total=total_batches) as pbar:
    for i in range(0, len(X_train), batch_size):
        X_batch = X_train.iloc[i:i+batch_size]
        y_batch = y_train.iloc[i:i+batch_size]
        model.fit(X_batch, y_batch)  # Обучение модели на текущем батче
        pbar.update(1)  # Обновление прогресс-бара для каждого пройденного батча

# Оценка качества модели на тестовом наборе
y_pred = model.predict(X_test)
f1 = f1_score(y_test, y_pred, average='weighted')
accuracy = model.score(X_test, y_test)
print("Accuracy:", accuracy)
print("f1-score:", f1)

 88%|██████████████████████████████████████████████████████████████████████▉          | 7/8 [1:39:18<14:11, 851.18s/it]


Accuracy: 0.6128
f1-score: 0.5968941243976842


___Уменьшение датафрема на основании ТОП-100 популрных тегов___

In [60]:
top = df['most_popular_tag'].value_counts()[:50].index.tolist()

In [33]:
filter = pd.DataFrame(top)

In [61]:
df['flag'] = df['most_popular_tag'].apply(lambda x: x in top)

C:\Users\Anton\AppData\Local\Temp\ipykernel_14704\3438692720.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['flag'] = df['most_popular_tag'].apply(lambda x: x in top)


In [64]:
filter_df = df[df.flag == True]

In [65]:
random_sample = filter_df.sample(n=50000, random_state=42)

In [68]:
random_sample

,url,date and time,tag,title,text,most_popular_tag,preprocessed_text,flag
1023725,https://lenta.ru/news/2010/05/05/farmville/,2010-05-05 20:36:00,Интернет и СМИ,"Создатели игры FarmVille оценили себя в 4,6 ми...","[Компания Zynga Game Network, разработавшая иг...",Интернет и СМИ,компания zynga game network разработавшая игру...,True
161473,https://lenta.ru/news/2010/04/22/borneo/,2010-04-22 21:28:00,Наука и техника,На Борнео нашли самое длинное в мире насекомое,[Ученые обнаружили на острове Борнео десятки н...,Наука и техника,ученые обнаружили острове борнео десятки новых...,True
733410,https://lenta.ru/news/2017/07/12/pipeline/,2017-12-07 17:53:00,Экономика,Турция и Израиль договорились о строительстве ...,[Турция и Израиль собираются до конца года под...,Экономика,турция израиль собираются конца года подписать...,True
327292,https://lenta.ru/news/2021/06/11/chizhov/,2021-11-06 22:15:00,Мир,Постпред России при ЕС заявил о «кознях» «Севе...,[Постоянный представитель России при Евросоюзе...,Мир,постоянный представитель россии евросоюзе ес в...,True
1057119,https://www.mk.ru/politics/2022/02/24/spiker-e...,2022-02-24 16:18:00,"['Евросоюз - ЕС', 'Россия', 'Украина', 'Донбас...",Спикер Еврокомиссии прокомментировал возможнос...,Спикер Еврокомиссии Петер Стано дал комментари...,Россия,спикер еврокомиссии петер стано дал комментари...,True
...,...,...,...,...,...,...,...,...
373552,https://lenta.ru/news/2011/10/04/law/,2011-04-10 22:21:00,Россия,Госдума приняла в первом чтении закон о кастра...,"[Депутаты Государственной Думы во вторник, 4 о...",Россия,депутаты государственной думы вторник октября ...,True
232406,https://lenta.ru/news/2010/06/10/voronezh/,2010-10-06 13:33:00,Россия,В Воронежской области раскрыта преступная груп...,[В Новохоперском районе Воронежской области со...,Россия,новохоперском районе воронежской области сотру...,True
1063487,https://www.mk.ru/economics/2014/10/30/novak-k...,2014-10-30 21:48:00,"['Евросоюз - ЕС', 'Украина', 'Россия']","Новак, как и Эттингер, надеется, что газовый к...",Министр энергетики РФ Александр Новак надеется...,Россия,министр энергетики рф александр новак надеется...,True
436536,https://lenta.ru/news/2022/05/05/curls/,2022-05-05 17:44:00,Ценности,Кудрявые прически названы главным трендом весны,[Стилисты назвали кудрявые прически главным тр...,Ценности,стилисты назвали кудрявые прически главным тре...,True


In [67]:
random_sample['most_popular_tag'].value_counts()

most_popular_tag
Россия                                             10362
Мир                                                 7236
Экономика                                           4020
Спорт                                               3448
Бывший СССР                                         3397
Наука и техника                                     2686
Культура                                            2419
Интернет и СМИ                                      2039
Силовые структуры                                   1526
Из жизни                                            1419
Среда обитания                                      1369
США                                                 1008
Москва                                               887
Бизнес                                               881
Ценности                                             875
Происшествия                                         827
Путешествия                                          812
Украина       

In [69]:
!python -m spacy download ru_core_news_md

     ---------------------------------------- 0.0/41.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/41.9 MB ? eta -:--:--
     --------------------------------------- 0.0/41.9 MB 388.9 kB/s eta 0:01:48
     --------------------------------------- 0.1/41.9 MB 573.4 kB/s eta 0:01:13
     ---------------------------------------- 0.2/41.9 MB 1.2 MB/s eta 0:00:34
     ---------------------------------------- 0.3/41.9 MB 1.6 MB/s eta 0:00:27
     ---------------------------------------- 0.4/41.9 MB 1.6 MB/s eta 0:00:26
      --------------------------------------- 0.9/41.9 MB 2.8 MB/s eta 0:00:15
     - -------------------------------------- 1.3/41.9 MB 3.5 MB/s eta 0:00:12
     - -------------------------------------- 1.7/41.9 MB 4.4 MB/s eta 0:00:10
     -- ------------------------------------- 2.2/41.9 MB 4.8 MB/s eta 0:00:09
     -- ------------------------------------- 2.7/41.9 MB 5.5 MB/s eta 0:00:08
     --- ------------------------------------ 3.2/41.9 MB 6.0 MB

In [70]:
nlp = spacy.load("ru_core_news_md")

In [156]:
def lemmatize(doc):
    words = []
    for token in doc:
        if (token.is_stop != True) and (token.is_punct != True) and\
            (token.is_space != True) and (token.is_digit != True):
            words.append(token.lemma_)
    return ' '.join(words)

In [157]:
processed_texts = []
for text in tqdm(random_sample['text'], desc="Lemmatizing"):
    doc = nlp(text)
    processed_texts.append(lemmatize(doc))

Lemmatizing: 100%|███████████████████████████████████████████████████████████████| 50000/50000 [54:58<00:00, 15.16it/s]


In [159]:
processed_df = pd.DataFrame({'processed_texts': processed_texts})

In [188]:
processed_df

,processed_texts
0,компания zynga game network разработать игра f...
1,учёный обнаружить остров борнео десяток новый ...
2,турция израиль собираться конец год подписать ...
3,постоянный представитель россия евросоюз ес вл...
4,спикер еврокомиссия петер стано дать комментар...
...,...
49995,депутат государственный дума вторник октябрь п...
49996,новохопёрский район воронежский область сотруд...
49997,министр энергетика рф александр новак надеятьс...
49998,стилист назвать кудрявый причёска главный трен...


In [191]:
random_sample['most_popular_tag']

1023725     Интернет и СМИ
161473     Наука и техника
733410           Экономика
327292                 Мир
1057119             Россия
                ...       
373552              Россия
232406              Россия
1063487             Россия
436536            Ценности
5039                   Мир
Name: most_popular_tag, Length: 50000, dtype: object

In [192]:
processed_df['most_popular_tag'] = random_sample['most_popular_tag'].reset_index(drop=True)

In [193]:
processed_df

,processed_texts,most_popular_tag
0,компания zynga game network разработать игра f...,Интернет и СМИ
1,учёный обнаружить остров борнео десяток новый ...,Наука и техника
2,турция израиль собираться конец год подписать ...,Экономика
3,постоянный представитель россия евросоюз ес вл...,Мир
4,спикер еврокомиссия петер стано дать комментар...,Россия
...,...,...
49995,депутат государственный дума вторник октябрь п...,Россия
49996,новохопёрский район воронежский область сотруд...,Россия
49997,министр энергетика рф александр новак надеятьс...,Россия
49998,стилист назвать кудрявый причёска главный трен...,Ценности


In [194]:
processed_df.to_csv('H:/Python311/Scripts/Файлы/drive-download-20240225T125924Z-002/preprocessed_text_50000.csv', sep=';', index = False)

In [4]:
preprocessed_df = pd.read_csv('H:/Python311/Scripts/Файлы/drive-download-20240225T125924Z-002/preprocessed_text_50000.csv', sep=';')

In [7]:
preprocessed_df

,processed_texts,most_popular_tag
0,компания zynga game network разработать игра f...,Интернет и СМИ
1,учёный обнаружить остров борнео десяток новый ...,Наука и техника
2,турция израиль собираться конец год подписать ...,Экономика
3,постоянный представитель россия евросоюз ес вл...,Мир
4,спикер еврокомиссия петер стано дать комментар...,Россия
...,...,...
49995,депутат государственный дума вторник октябрь п...,Россия
49996,новохопёрский район воронежский область сотруд...,Россия
49997,министр энергетика рф александр новак надеятьс...,Россия
49998,стилист назвать кудрявый причёска главный трен...,Ценности


In [6]:
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")

In [18]:
tfidf_vectorizer = TfidfVectorizer(token_pattern=r'(?<!\d)\b\w+\b(?!\d)', ngram_range=(1, 1), stop_words=russian_stopwords, min_df=3)

In [19]:
tfidf_vectorizer_text = tfidf_vectorizer.fit(preprocessed_df['processed_texts'])

In [20]:
# Преобразование текста в векторы TF-IDF
tfidf_vectors = tfidf_vectorizer_text.transform(preprocessed_df['processed_texts'])

In [21]:
tfidf_vectorizer.get_feature_names_out().tolist()

['0',
 '00',
 '000',
 '001',
 '0041',
 '01',
 '02',
 '03',
 '04',
 '05',
 '054',
 '057',
 '06',
 '07',
 '08',
 '09',
 '099',
 '1',
 '10',
 '100',
 '1000',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '11',
 '110',
 '1100',
 '111',
 '112',
 '112в',
 '113',
 '114',
 '1143',
 '115',
 '116',
 '117',
 '118',
 '119',
 '11b',
 '12',
 '120',
 '121',
 '122',
 '123',
 '124',
 '125',
 '126',
 '127',
 '128',
 '129',
 '13',
 '130',
 '130j',
 '131',
 '132',
 '133',
 '134',
 '135',
 '135w',
 '136',
 '137',
 '138',
 '139',
 '14',
 '140',
 '141',
 '142',
 '143',
 '144',
 '145',
 '146',
 '147',
 '148',
 '149',
 '15',
 '150',
 '151',
 '152',
 '153',
 '154',
 '154м',
 '155',
 '156',
 '157',
 '158',
 '159',
 '15e',
 '15se',
 '16',
 '160',
 '160м',
 '161',
 '162',
 '163',
 '164',
 '165',
 '166',
 '167',
 '168',
 '169',
 '16a',
 '16c',
 '17',
 '170',
 '171',
 '171ш',
 '172',
 '173',
 '174',
 '175',
 '176',
 '177',
 '178',
 '179',
 '17a',
 '17д',
 '18',
 '180',
 '1800',
 '181',
 '1

In [25]:
most_popular_tags = preprocessed_df['most_popular_tag'].values.reshape(-1, 1)
most_popular_tags

array([['Интернет и СМИ'],
       ['Наука и техника'],
       ['Экономика'],
       ...,
       ['Россия'],
       ['Ценности'],
       ['Мир']], dtype=object)

In [22]:
tfidf_vectors[:5].toarray().shape

(5, 50215)

In [27]:
import numpy as np
import pandas as pd
from tqdm import tqdm

# Путь к CSV файлу, куда будут добавляться данные
csv_path = 'H:/Python311/Scripts/Файлы/drive-download-20240225T125924Z-002/tfidf_vectors_tags_batch_50000.csv'

# Размер батча
batch_size = 1000

# Количество строк
num_rows = tfidf_vectors.shape[0]

# Количество батчей
num_batches = (num_rows - 1) // batch_size + 1

# Цикл по всем батчам
for i in tqdm(range(num_batches), desc="Processing batches"):
    # Определение индексов текущего батча
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, num_rows)
    
    # Получение TF-IDF векторов для текущего батча
    tfidf_vectors_batch = tfidf_vectors[start_idx:end_idx]
    
    # Преобразование разреженной матрицы в плотную матрицу numpy
    dense_batch = tfidf_vectors_batch.toarray()
    
    # Получение значений 'most_popular_tag' для текущего батча
    most_popular_tags_batch = preprocessed_df['most_popular_tag'].iloc[start_idx:end_idx].values.reshape(-1, 1)
    
    # Объединение TF-IDF векторов с 'most_popular_tag'
    batch_data = np.concatenate((dense_batch, most_popular_tags_batch), axis=1)
    
    # Запись батча в CSV файл
    if i == 0:
        # Если это первый батч, записываем его с заголовком
        pd.DataFrame(batch_data, columns=np.append(tfidf_vectorizer_text.get_feature_names_out(), 'most_popular_tag')).to_csv(csv_path, mode='a', header=True, index=False)
    else:
        # Если это последующие батчи, записываем их без заголовка
        pd.DataFrame(batch_data).to_csv(csv_path, mode='a', header=False, index=False)


Processing batches: 100%|████████████████████████████████████████████████████████████| 50/50 [1:10:55<00:00, 85.11s/it]


In [29]:
tfidf_vectors_tags = pd.read_csv('H:/Python311/Scripts/Файлы/drive-download-20240225T125924Z-002/tfidf_vectors_tags_batch_50000.csv')

MemoryError: 

In [205]:
'''
Алгоритм сохранения векторов без целевой переменной
'''

# import numpy as np

# # Путь к CSV файлу, куда будут добавляться данные
# csv_path = 'H:/Python311/Scripts/Файлы/drive-download-20240225T125924Z-002/tfidf_vectors_batch_50000.csv'

# # Размер батча
# batch_size = 1000

# # Количество строк
# num_rows = tfidf_vectors.shape[0]

# # Количество батчей
# num_batches = (num_rows - 1) // batch_size + 1

# # Цикл по всем батчам
# for i in tqdm(range(num_batches), desc="Processing batches"):
#     # Определение индексов текущего батча
#     start_idx = i * batch_size
#     end_idx = min((i + 1) * batch_size, num_rows)
    
#     # Получение TF-IDF векторов для текущего батча
#     tfidf_vectors_batch = tfidf_vectors[start_idx:end_idx]
    
#     # Преобразование разреженной матрицы в плотную матрицу numpy
#     dense_batch = tfidf_vectors_batch.toarray()
    
#     # Добавление батча в CSV файл
#     if i == 0:
#         # Если это первый батч, записываем его с заголовком
#         pd.DataFrame(dense_batch, columns = tfidf_vectorizer.get_feature_names_out()).to_csv(csv_path, mode='a', header=True, index=False)
#     else:
#         # Если это последующие батчи, записываем их без заголовка
#         pd.DataFrame(dense_batch).to_csv(csv_path, mode='a', header=False, index=False)

Processing batches: 100%|███████████████████████████████████████████████████████████| 50/50 [4:30:54<00:00, 325.08s/it]


In [211]:
X_train, X_test, y_train, y_test = train_test_split(pd.read_csv('H:/Python311/Scripts/Файлы/drive-download-20240225T125924Z-002/tfidf_vectors_tags_batch_50000.csv').drop('most_popular_tag'), processed_df['most_popular_tag'], test_size=0.2, random_state=42)

MemoryError: 

In [ ]:
# Создание и обучение модели

model = GradientBoostingClassifier()
batch_size = 5000

total_batches = len(X_train) // batch_size + 1
unique_tags = y.unique().tolist()

with tqdm(total=total_batches) as pbar:
    for i in range(0, len(X_train), batch_size):
        X_batch = X_train.iloc[i:i+batch_size]
        y_batch = y_train.iloc[i:i+batch_size]
        model.fit(X_batch, y_batch)  # Обучение модели на текущем батче
        pbar.update(1)  # Обновление прогресс-бара для каждого пройденного батча

# Оценка качества модели на тестовом наборе
y_pred = model.predict(X_test)
f1 = f1_score(y_test, y_pred, average='weighted')
accuracy = model.score(X_test, y_test)
print("Accuracy:", accuracy)
print("f1-score:", f1)

In [204]:
csv_path = 'H:/Python311/Scripts/Файлы/drive-download-20240225T125924Z-002/tfidf_vectors_batch_50000.csv'
batch_size = 5000

# Количество строк
num_rows = tfidf_vectors.shape[0]

# Количество батчей
num_batches = (num_rows - 1) // batch_size + 1

# Загрузка данных из csv
reader = pd.read_csv(csv_path, chunksize=batch_size)

# Создание модели градиентного бустинга
model = GradientBoostingClassifier()

# Цикл по всем батчам
for i in tqdm(range(num_batches), desc="Processing batches"):
    # Определение индексов текущего батча
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, num_rows)
    X 
        
        
        # Разделение данных на трейн и тест
        X = batch_data.drop(columns=['id'])  # Предположим, что 'id' - это идентификатор строки
        y = processed_df.loc[processed_df['id'].isin(batch_data['id']), 'most_popular_tag']
        
        
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Обучение модели на трейне
        model.fit(X_train, y_train)

        # Предсказание на тесте
        y_pred = model.predict(X_test)

        # Оценка модели
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')

        # Обновление прогресса
        pbar.update(1)
        pbar.set_postfix({"Accuracy": accuracy, "F1-score": f1})

        pbar.set_postfix({"Accuracy": accuracy, "F1-score": f1})
   

MemoryError: Unable to allocate 61.2 GiB for an array with shape (50000, 164181) and data type float64

In [128]:
a = pd.read_csv('H:/Python311/Scripts/Файлы/drive-download-20240225T125924Z-002/preprocessed_text.csv', sep=',')

In [142]:
a

,processed_text
0,глава аджария аслан абашидзе принять решение у...
1,июль российский железный дорога начать продава...
2,использование биткойнов оплата товар услуга да...
3,число официально подтвердить случай заражение ...
4,президент россия владимир путин телефон обсуди...
...,...
1271186,облачная прояснениями погода ожидаться петербу...
1271187,медиапросторам разойтись видео задержание нача...
1271188,польский сторона граница украина застрять выех...
1271189,президент россия владимир путин следующий неде...
